In [91]:
import pickle
import numpy as np
from datasets import load_dataset
import tqdm
import math
import sys

In [92]:
# # GLUE
# dataset = load_dataset("glue", "sst2")
# val_dataset = dataset["validation"]

dataset = load_dataset("christophsonntag/OLID")
train_dataset = dataset["train"]
test_dataset = dataset["test"]

### CNN Based Classifier

In [93]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [104]:
# Define the CNN architecture for 1D vectors
class CNNClassifier(nn.Module):
    def __init__(self):
        super(CNNClassifier, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 32, kernel_size = 3)
#         self.conv2 = nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3)
#         self.conv3 = nn.Conv1d(in_channels = 32, out_channels = 16, kernel_size = 3)
#         self.pool = nn.MaxPool1d(kernel_size = 2)
        self.fc1 = nn.Linear(2 * 768, 4)  # Adjust input size according to your data
        self.fc2 = nn.Linear(4, 1)      # Output layer with 1 neuron for binary classification

    def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = self.pool(nn.functional.relu(self.conv3(x)))
#         x = x.view(-1, 16 * 94)  # Adjust the size according to the output size of the last convolutional layer
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

In [105]:
# Sample data loader for demonstration
class SampleDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [106]:
# Get Training Data

# Read pkl file 
with open('embeddings/OLID-MPNET/OLID-concat-train-768.pkl', 'rb') as f:
    vectors = pickle.load(f)
    
X = np.array(vectors)

labels = []
for item in train_dataset:
    if item['subtask_a'] == 'NOT':
        label = 0
    else:
        label = 1
    labels.append(label)
    
Y = np.array(labels)

### Determining which Subset 

In [107]:
# Read Instance Scores
with open('scores/OLID-MPNET/instance-scores-concat-OLID-L1-30-40-30.pkl', 'rb') as f:
    instance_scores = pickle.load(f)
    
# Sort indices based on the score in descending order
def sort_indices_by_values(values):
    return sorted(range(len(values)), key=lambda i: np.abs(values[i]), reverse = True)

# Example usage:
sorted_indices = sort_indices_by_values(instance_scores)

In [108]:
# Compute subset length

percentage = 0.75
name = 'models/OLID/temp.pth' 

subset_len = int(percentage * len(X))
sorted_subset_indices = sorted_indices[:subset_len]

In [109]:
X_eff = []
Y_eff = []

for i in sorted_subset_indices:
    X_eff.append(X[i])
    Y_eff.append(Y[i])

In [110]:
N = len(X_eff)
X_train = torch.from_numpy(np.array(X_eff).reshape(N, 1, 2 * 768))
Y_train = torch.from_numpy(np.array(Y_eff).reshape(N, 1, 1)).float()
print(str(N) + ' samples selected')

9930 samples selected


In [111]:
c0 = 0
c1 = 0
for i in Y_eff:
    if i == 0:
        c0 += 1
    else:
        c1 += 1
        
print('Class 0: ' + str(c0))
print('Class 1: ' + str(c1))

Class 0: 6511
Class 1: 3419


### Start Training

In [113]:
# Initialize the model, loss function, and optimizer
model = CNNClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)

# Create data loader
trainset = SampleDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle=True)

# Train the model
for epoch in tqdm.tqdm(range(50)):  # Adjust number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 9:  # Print every 10 mini-batches
            print('[%d, %5d] loss: %.4f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

  0%|          | 0/50 [00:00<?, ?it/s]

[1,    10] loss: 0.0785
[1,   110] loss: 0.7076


  2%|▏         | 1/50 [00:00<00:19,  2.53it/s]

[1,   210] loss: 0.6407
[1,   310] loss: 0.6209
[2,    10] loss: 0.0643
[2,   110] loss: 0.5963


  4%|▍         | 2/50 [00:00<00:12,  3.70it/s]

...

[48,   110] loss: 0.4400
[48,   210] loss: 0.4629
[48,   310] loss: 0.4553
[49,    10] loss: 0.0481
[49,   110] loss: 0.4532


 98%|█████████▊| 49/50 [00:17<00:00,  2.81it/s]

[49,   210] loss: 0.4490
[49,   310] loss: 0.4518
[50,    10] loss: 0.0491
[50,   110] loss: 0.4413


100%|██████████| 50/50 [00:17<00:00,  2.88it/s]

[50,   210] loss: 0.4592
[50,   310] loss: 0.4514
Finished Training


In [115]:
# Save the trained model
torch.save(model.state_dict(), name)

# Perform inference with the saved model
loaded_model = CNNClassifier()
loaded_model.load_state_dict(torch.load(name))
loaded_model.eval()

CNNClassifier(
  (fc1): Linear(in_features=1536, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=1, bias=True)
)

In [117]:
# Lets do testing

# Example inference
# sample_input = torch.randn(1, 1, 768)  # Example input
# output = loaded_model(sample_input)
# print("Model output:", output.item())

# Read pkl file 
with open('embeddings/OLID-MPNET/OLID-concat-test-768.pkl', 'rb') as f:
    test_vectors = pickle.load(f)

In [119]:
gt = []
preds = []

for i in tqdm.tqdm(range(len(test_vectors))):
    input_vector = torch.tensor(test_vectors[i].reshape(1, 1, 2 * 768))
    label = test_dataset[i]['subtask_a']
    if label == 'NOT':
        actual = 0
    else:
        actual = 1
    output = loaded_model(input_vector) 
    if output < 0.5:
        pred = 0
    else:
        pred = 1
    gt.append(actual)
    preds.append(pred)

100%|██████████| 860/860 [00:00<00:00, 10095.50it/s]


In [120]:
# Print the number of misclassifications
misc = 0
for i in range(len(gt)):
    if gt[i] != preds[i]:
        misc += 1

print(misc)

157


In [121]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(gt, preds)

print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.95      0.88       620
           1       0.77      0.49      0.60       240

    accuracy                           0.82       860
   macro avg       0.80      0.72      0.74       860
weighted avg       0.81      0.82      0.80       860

